In [249]:
# Imports
import numpy as np
import pandas as pd
import random
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.metrics import accuracy_score, log_loss, make_scorer

random.seed(1)

root = "/home/austin/Github/kaggle-ncaa-2018/"

data = pd.read_csv(root + "derived_data/Master.csv")
matchups = pd.read_csv("/home/austin/Github/DataFiles/TourneyWinratesBySeed.csv")
tour_ratios = pd.read_csv(root + "derived_data/ratios/NCAATourneyDetailedResultsRatios.csv")

In [250]:
seasons = range(2003, 2018)
seasons_test = [2017]
seasons_train = [season for season in seasons if season not in seasons_test]

all_columns = data.columns.tolist()
non_stats_columns = ["Season", "TeamID", "Seed", "Elo"]#, "BPI", "Predictor_Score"]
stats_columns = [c for c in all_columns if c not in non_stats_columns]

In [251]:
tourney_games = tour_ratios.loc[tour_ratios["Season"].isin(seasons)]
tourney_games = tourney_games.reset_index(drop=True)
extra_cols = ["NumTeamSeed", "DenTeamSeed", "NumTeamElo", "DenTeamElo", "HistWinPct"]

pred_data = pd.DataFrame(index=range(tourney_games.shape[0]), columns= ["Season"] + stats_columns + extra_cols + ["NumTeamWon"])
pred_data.loc[:,"NumTeamWon"] = np.ones(tourney_games.shape[0])
index = 0
for i in range(len(seasons)):
    season = seasons[i]
    tourney_games_for_season = tourney_games.loc[tourney_games["Season"] == season].reset_index()
    for j, row in tourney_games_for_season.iterrows():
        pred_data.loc[index, "Season"] = season
        teamA_id = tourney_games_for_season.at[j, "WTeamID"]
        teamB_id = tourney_games_for_season.at[j, "LTeamID"]
        
        teamA_seed = data.loc[(data["TeamID"] == teamA_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "Seed"]
        teamB_seed = data.loc[(data["TeamID"] == teamB_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "Seed"]
        
        teamA_elo = data.loc[(data["TeamID"] == teamA_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "Elo"]
        teamB_elo = data.loc[(data["TeamID"] == teamB_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "Elo"]
        
        teamA_stats = data.loc[(data["TeamID"] == teamA_id) & (data["Season"] == season), stats_columns].as_matrix()
        teamB_stats = data.loc[(data["TeamID"] == teamB_id) & (data["Season"] == season), stats_columns].as_matrix()
        
#         teamA_bpi = data.loc[(data["TeamID"] == teamA_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "BPI"]
#         teamB_bpi = data.loc[(data["TeamID"] == teamB_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "BPI"]
        
#         teamA_pred = data.loc[(data["TeamID"] == teamA_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "Predictor_Score"]
#         teamB_pred = data.loc[(data["TeamID"] == teamB_id) & (data["Season"] == season)].reset_index(drop=True).loc[0, "Predictor_Score"]
        
        r = random.random()
        
        if r > 0.5:
            pred_data.at[index, stats_columns] = (teamA_stats / teamB_stats).ravel()
            pred_data.at[index, "NumTeamSeed"] = teamA_seed
            pred_data.at[index, "DenTeamSeed"] = teamB_seed
            pred_data.at[index, "NumTeamElo"] = teamA_elo
            pred_data.at[index, "DenTeamElo"] = teamB_elo
            pred_data.at[index, "HistWinPct"] = matchups[(matchups["WinSeed"] == teamA_seed) & (matchups["LoseSeed"] == teamB_seed)]["1985"]
#             pred_data.at[index, "NumTeamBPI"] = teamA_bpi
#             pred_data.at[index, "DenTeamBPI"] = teamB_bpi
#             pred_data.at[index, "NumTeamPred"] = teamA_pred
#             pred_data.at[index, "DenTeamPred"] = teamB_pred
        else:
            pred_data.at[index, stats_columns] = (teamB_stats / teamA_stats).ravel()
            pred_data.at[index, "NumTeamSeed"] = teamB_seed
            pred_data.at[index, "DenTeamSeed"] = teamA_seed
            pred_data.at[index, "NumTeamElo"] = teamB_elo
            pred_data.at[index, "DenTeamElo"] = teamA_elo
            pred_data.at[index, "HistWinPct"] = matchups[(matchups["WinSeed"] == teamB_seed) & (matchups["LoseSeed"] == teamA_seed)]["1985"]
#             pred_data.at[index, "NumTeamBPI"] = teamB_bpi
#             pred_data.at[index, "DenTeamBPI"] = teamA_bpi
#             pred_data.at[index, "NumTeamPred"] = teamB_pred
#             pred_data.at[index, "DenTeamPred"] = teamA_pred
            pred_data.at[index, "NumTeamWon"] = 0
        
        index += 1

pred_data = pred_data.dropna(axis=0, how='any')

In [252]:
all_train_columns = stats_columns + extra_cols
train_columns_to_drop = ["FT_PCT", "FGM3", "FGA3", "FTM", "DEF_REB_PCT", "FTA", "FT_RATE", "4Factor",
                         "DR", "FG_PCT", "Stl", "ASSIST_RATIO", "PF", "TO", "Ast", "FGA", "OR"]

# train_columns_to_drop = []

train_columns = [c for c in all_train_columns if c not in train_columns_to_drop]

In [253]:
train_x = pred_data.loc[pred_data["Season"].isin(seasons_train)][train_columns].as_matrix()
train_y = pred_data.loc[pred_data["Season"].isin(seasons_train)]["NumTeamWon"].as_matrix()
test_x = pred_data.loc[pred_data["Season"].isin(seasons_test)][train_columns].as_matrix()
test_y = pred_data.loc[pred_data["Season"].isin(seasons_test)]["NumTeamWon"].as_matrix()

In [254]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# parameters = {'max_features': np.arange(1,12,1),
#               'min_samples_split': np.arange(2,50,2),
#               'min_samples_leaf': np.arange(1,15,1)
#               }

# clf = RandomizedSearchCV(ExtraTreesRegressor(n_estimators=1000, n_jobs=-1, random_state=0), parameters,
#                    cv=10, scoring=make_scorer(log_loss), n_iter=50)

# clf = ExtraTreesRegressor(n_estimators=1000, n_jobs=-1, random_state=0, max_features=4, min_samples_split=40)

# clf.fit(train_x, train_y.ravel())

In [255]:
# preds_train = clf.predict(train_x)
# preds_test = clf.predict(test_x)

# print(clf.best_params_)

# # print("Accuracy, Train: ", accuracy_score(train_y, preds_train > 0.5))
# # print("Log Loss, Train: ", log_loss(train_y, preds_train))
# # print("")
# print("Accuracy, Test:  ", accuracy_score(test_y, preds_test > 0.5))
# print("Log Loss, Test:  ", log_loss(test_y, preds_test))

In [260]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

parameters = {'max_features': np.arange(1,12,1),
              'min_samples_split': np.arange(2,50,4),
              'min_samples_leaf': np.arange(1,15,1)
              }

clf1 = RandomizedSearchCV(RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0), parameters,
                    cv=10, scoring=make_scorer(log_loss), n_iter=20)
# clf1 = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=0, max_features=5, min_samples_split=40)

# clf = ExtraTreesRegressor(n_estimators=1000, max_features=1, n_jobs=-1)
# clf = LogisticRegression(n_jobs=-1)
clf1.fit(train_x, train_y.ravel())
# print("OOB Score: ", clf.oob_score_)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
           oob_score=False, random_state=0, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'min_samples_split': array([ 2,  6, 10, 14, 18, 22, 26, 30, 34, 38, 42, 46]), 'max_features': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]), 'min_samples_leaf': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=make_scorer(log_loss),
          verbose=0)

In [261]:
preds_train1 = clf1.predict(train_x)
preds_test1 = clf1.predict(test_x)

# threshold = .1

# preds_test1[preds_test1 > 1 - threshold] = 1
# preds_test1[preds_test1 < threshold] = 0

print(clf1.best_params_)

print("Accuracy, Train: ", accuracy_score(train_y, preds_train1 > 0.5))
print("Log Loss, Train: ", log_loss(train_y, preds_train1))
# print("")
print("Accuracy, Test:  ", accuracy_score(test_y, preds_test1 > 0.5))
print("Log Loss, Test:  ", log_loss(test_y, preds_test1))

variables = train_columns
feature_importance = clf1.feature_importances_

feature_importance, variables = (list(t) for t in zip(*sorted(zip(feature_importance, variables))))

for i in range(len(feature_importance)):
    print(variables[i], ":", feature_importance[i])

{'min_samples_split': 10, 'max_features': 10, 'min_samples_leaf': 1}
Accuracy, Train:  0.9781181619256017
Log Loss, Train:  0.241729193049447
Accuracy, Test:   0.746268656716418
Log Loss, Test:   0.5581382700221917


4FACTOR : 0.02830725266323982
Blk : 0.035794866139219214
DEF_EFF : 0.03627180253409459
OFF_REB_PCT : 0.03844534293907675
FGM : 0.039907943916444394
TURNOVER_RATE : 0.04065380783449319
WINPCT : 0.04288766435369602
DenTeamSeed : 0.04576789430324574
OFF_EFF : 0.047571705193758106
NumTeamSeed : 0.06932996407865333
PIE : 0.07303979017973293
DenTeamElo : 0.07644663015593192
NumTeamElo : 0.13482640634995546
HistWinPct : 0.2907489293584585
